In [22]:
import pandas as pd
from pymer4.models import Lmer
from sklearn.preprocessing import StandardScaler

In [23]:
# Cargar los datos
df = pd.read_csv('../data/retail_price.csv')

# Convertir variables categóricas
df['product_category_name'] = df['product_category_name'].astype('category')
df['month_year'] = pd.to_datetime(df['month_year'], format='%d-%m-%Y')

# Escalar las variables predictoras para mejorar la convergencia
scaler = StandardScaler()
df_transformed = df.copy()
df_transformed[['unit_price', 'product_score']] = scaler.fit_transform(df[['unit_price', 'product_score']])

display(df)

,product_id,product_category_name,month_year,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,...,comp_1,ps1,fp1,comp_2,ps2,fp2,comp_3,ps3,fp3,lag_price
0,bed1,bed_bath_table,2017-05-01,1,45.95,15.100000,45.950000,39,161,2,...,89.9,3.9,15.011897,215.000000,4.4,8.760000,45.95,4.0,15.100000,45.900000
1,bed1,bed_bath_table,2017-06-01,3,137.85,12.933333,45.950000,39,161,2,...,89.9,3.9,14.769216,209.000000,4.4,21.322000,45.95,4.0,12.933333,45.950000
2,bed1,bed_bath_table,2017-07-01,6,275.70,14.840000,45.950000,39,161,2,...,89.9,3.9,13.993833,205.000000,4.4,22.195932,45.95,4.0,14.840000,45.950000
3,bed1,bed_bath_table,2017-08-01,4,183.80,14.287500,45.950000,39,161,2,...,89.9,3.9,14.656757,199.509804,4.4,19.412885,45.95,4.0,14.287500,45.950000
4,bed1,bed_bath_table,2017-09-01,2,91.90,15.100000,45.950000,39,161,2,...,89.9,3.9,18.776522,163.398710,4.4,24.324687,45.95,4.0,15.100000,45.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,bed5,bed_bath_table,2017-05-01,1,215.00,8.760000,215.000000,56,162,5,...,89.9,3.9,15.011897,215.000000,4.4,8.760000,45.95,4.0,15.100000,214.950000
672,bed5,bed_bath_table,2017-06-01,10,2090.00,21.322000,209.000000,56,162,5,...,89.9,3.9,14.769216,209.000000,4.4,21.322000,45.95,4.0,12.933333,215.000000
673,bed5,bed_bath_table,2017-07-01,59,12095.00,22.195932,205.000000,56,162,5,...,89.9,3.9,13.993833,205.000000,4.4,22.195932,45.95,4.0,14.840000,209.000000
674,bed5,bed_bath_table,2017-08-01,52,10375.00,19.412885,199.509804,56,162,5,...,89.9,3.9,14.656757,199.509804,4.4,19.412885,45.95,4.0,14.287500,205.000000


In [24]:
# Definir y ajustar un modelo linear mixto
model = Lmer(
    formula='qty ~ unit_price + product_score + holiday + (1 + unit_price | product_category_name)',
    data=df_transformed
)

# Ajustar el modelo y mostrar resultados
fit = model.fit()

display(fit)

Linear mixed model fit by REML [’lmerMod’]
Formula: qty~unit_price+product_score+holiday+(1+unit_price|product_category_name)

Family: gaussian	 Inference: parametric

Number of observations: 676	 Groups: {'product_category_name': 9.0}

Log-likelihood: -2779.448 	 AIC: 5574.896

Random effects:

                              Name      Var     Std
product_category_name  (Intercept)   12.693   3.563
product_category_name   unit_price   36.819   6.068
Residual                            214.787  14.656

                               IV1         IV2   Corr
product_category_name  (Intercept)  unit_price  0.912

Fixed effects:



/home/andres/Documentos/univ/maestria/4to/del-negociante/.venv/lib/python3.13/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/andres/Documentos/univ/maestria/4to/del-negociante/.venv/lib/python3.13/site-packages/pymer4/models/Lmer.py:744: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_corrs = ran_corrs.applymap(


,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),9.619,6.254,12.984,1.717,7.653,5.603,0.001,***
unit_price,-0.988,-5.617,3.641,2.362,6.736,-0.418,0.689,
product_score,0.153,-1.087,1.392,0.633,90.613,0.241,0.810,
holiday,3.643,2.466,4.821,0.601,652.251,6.064,0.000,***


In [25]:
# Análisis 1: Efecto de los feriados
holiday_effect = df.groupby('holiday').agg(
    Ventas_promedio=('qty', 'mean'),
    Precio_promedio=('unit_price', 'mean')
).reset_index()

display(holiday_effect)

,holiday,Ventas_promedio,Precio_promedio
0,0,11.285714,97.013803
1,1,12.784974,106.008520
2,2,14.884146,111.996015
3,3,22.450000,100.167249
4,4,23.886364,105.089355


In [27]:
# Análisis 2: Sensibilidad al precio por categoría
category_price_effect = df.groupby('product_category_name', observed=True).agg(
    Ventas_promedio=('qty', 'mean'),
    Sensibilidad_Precio=('unit_price', lambda x: x.corr(df.loc[x.index, 'qty'])),
    Participacion_mercado=('qty', 'sum')
).reset_index()

display(category_price_effect)

,product_category_name,Ventas_promedio,Sensibilidad_Precio,Participacion_mercado
0,bed_bath_table,16.819672,0.371125,1026
1,computers_accessories,16.913043,0.069867,1167
2,consoles_games,10.863636,-0.566361,239
3,cool_stuff,9.789474,-0.101479,558
4,furniture_decor,18.625000,0.101281,894
5,garden_tools,14.987500,-0.297984,2398
6,health_beauty,14.169231,-0.186943,1842
7,perfumery,9.384615,-0.258426,244
8,watches_gifts,13.893204,-0.241066,1431


In [28]:
# Análisis 3: Impacto del score de producto
score_effect = df.groupby('product_score').agg(
    Ventas_promedio=('qty', 'mean'),
    Precio_promedio=('unit_price', 'mean')
).reset_index()

display(score_effect)

,product_score,Ventas_promedio,Precio_promedio
0,3.3,10.090909,92.101364
1,3.5,17.222222,93.016403
2,3.7,11.680000,165.840942
3,3.8,11.085106,121.385994
4,3.9,15.845070,66.749108
5,4.0,12.714286,82.124533
6,4.1,19.400000,102.212118
7,4.2,15.219355,115.924347
8,4.3,11.166667,126.003535
9,4.4,13.571429,110.205048


In [29]:
# Exportar resultados para Power BI
holiday_effect.to_csv('holiday_effect.csv', index=False)
category_price_effect.to_csv('category_analysis.csv', index=False)
score_effect.to_csv('score_analysis.csv', index=False)